In [1]:
# Author: James E T Smith <james.smith9113@gmail.com>
# Date: May 15th, 2018

# General
import warnings
warnings.filterwarnings('ignore')

# Import PySCF modules and make molecule object
from pyscf import gto, scf, mcscf
from pyscf.shciscf import shci

mol = gto.Mole()
mol.atom = '''
  C        0.00000        1.40272        0.00000
  H        0.00000        2.49029        0.00000
  C       -1.21479        0.70136        0.00000
  H       -2.15666        1.24515        0.00000
  C       -1.21479       -0.70136        0.00000
  H       -2.15666       -1.24515        0.00000
  C        0.00000       -1.40272        0.00000
  H        0.00000       -2.49029        0.00000
  C        1.21479       -0.70136        0.00000
  H        2.15666       -1.24515        0.00000
  C        1.21479        0.70136        0.00000
  H        2.15666        1.24515        0.00000'''
mol.basis = 'ccpvdz'
mol.symmetry = True
mol.build()
mol.nelec # Show the number of \alpha and \beta electrons

(21, 21)

In [2]:
# Make RHF Object and optimize orbitals
mf = scf.RHF(mol).run()

converged SCF energy = -230.721010772091


In [3]:
# Save orbitals so we can examine them with 3 party program (we use Jmol most often)
from pyscf.tools import molden
with open('benzene.molden','w') as f:
    molden.header(mol, f)
    molden.orbital_coeff(mol, f, mf.mo_coeff, ene=mf.mo_energy, occ=mf.mo_occ)



# Jmol Instructions:
- Open 'benzene.molden' in Jmol
- Go to File > Console to open the scripting console
- Enter the following two commands:
    1. `mo nomesh`
    2. `mo fill`
- Now you can enter `mo i` where is the number of the orbital, if you choose 21 (HOMO) then you should see something like the image below
<img src="benzene_mo_first_look.jpg" alt="benzene_mo_first_look" title="benzene_mo_first_look">
- In Jmol we can see the symmetry and occupation which can be very helpful when picking an active space


# Picking an Active Space
- Picking the valence orbitals is a common tactic in MCSCF calculation
- When aromatic systems are involved we often choose the conjugated 2p orbitals that contribute to the aromaticity
- __Our goal here is to select the 6 bonding and anti-bonding 2pz orbitals__
- See if you can find them all by visualizing the MOs in jmol


| | | |
|:-------------------------:|:-------------------------:|:-------------------------:|
|<img src="benzene_mo_17.jpg" alt="mo_17" title="MO_17">| <img src="benzene_mo_20.jpg" alt="mo_20" title="MO_20"> | <img src="benzene_mo_21.jpg" alt="mo_21" title="MO_21">|
|<img src="benzene_mo_22.jpg" alt="mo_22" title="MO_22">| <img src="benzene_mo_23.jpg" alt="mo_23" title="MO_23"> | <img src="benzene_mo_30.jpg" alt="mo_30" title="MO_30">|

# Back to PySCF
- Since we know number of the MOs we want to put in the active space we can specify it explicitly in PySCF
- When running research level calculations we suggest that you increase the verbosity to at least 5, it can be critical in debugging and detecting errors in the calculation

In [4]:
ncas = 6
nelecas = 6
mc = shci.SHCISCF(mf, ncas, nelecas)
mo = mcscf.sort_mo( mc, mf.mo_coeff, [17,20,21,22,23,30])
mc.verbose = 4 # You can increase this to see more of the output from PySCF
mc.mc2step(mo)[0]


******** <class 'pyscf.mcscf.mc1step_symm.SymAdaptedCASSCF'> flags ********
CAS (3e+3e, 6o), ncore = 18, nvir = 90
max_cycle_macro = 50
max_cycle_micro = 4
conv_tol = 1e-07
conv_tol_grad = None
orbital rotation max_stepsize = 0.02
augmented hessian ah_max_cycle = 30
augmented hessian ah_conv_tol = 1e-12
augmented hessian ah_linear dependence = 1e-14
augmented hessian ah_level shift = 0
augmented hessian ah_start_tol = 2.5
augmented hessian ah_start_cycle = 3
augmented hessian ah_grad_trust_region = 3
kf_trust_region = 3
kf_interval = 4
ci_response_space = 4
ci_grad_trust_region = 3
with_dep4 0
natorb = False
canonicalization = True
sorting_mo_energy = False
chkfile = /tmp/tmpErxsYF
max_memory 4000 MB (current use 258 MB)
internal_rotation = False
******** SHCI flags ********
executable = /home/james/Documents/Apps/Dice/Dice
mpiprefix = 
scratchDirectory = /tmp/
integralFile = ./FCIDUMP
configFile = ./input.dat
outputFile = ./output.dat
maxIter = 6
nPTiter = 0
Stochastic = True
num_thr

-230.79407523958318

# What if we just you HOMO-2 up to LUMO+2?

In [5]:
ncas = 6
nelecas = 6
mc = shci.SHCISCF(mf, ncas, nelecas)
mo = mcscf.sort_mo( mc, mf.mo_coeff, [19,20,21,22,23,24])
mc.verbose = 4 # You can increase this to see more of the output from PySCF
mc.mc2step(mo)[0]


******** <class 'pyscf.mcscf.mc1step_symm.SymAdaptedCASSCF'> flags ********
CAS (3e+3e, 6o), ncore = 18, nvir = 90
max_cycle_macro = 50
max_cycle_micro = 4
conv_tol = 1e-07
conv_tol_grad = None
orbital rotation max_stepsize = 0.02
augmented hessian ah_max_cycle = 30
augmented hessian ah_conv_tol = 1e-12
augmented hessian ah_linear dependence = 1e-14
augmented hessian ah_level shift = 0
augmented hessian ah_start_tol = 2.5
augmented hessian ah_start_cycle = 3
augmented hessian ah_grad_trust_region = 3
kf_trust_region = 3
kf_interval = 4
ci_response_space = 4
ci_grad_trust_region = 3
with_dep4 0
natorb = False
canonicalization = True
sorting_mo_energy = False
chkfile = /tmp/tmpmUZwqe
max_memory 4000 MB (current use 258 MB)
internal_rotation = False
******** SHCI flags ********
executable = /home/james/Documents/Apps/Dice/Dice
mpiprefix = 
scratchDirectory = /tmp/
integralFile = ./FCIDUMP
configFile = ./input.dat
outputFile = ./output.dat
maxIter = 6
nPTiter = 0
Stochastic = True
num_thr

-230.76255882363014

# Approximations Using the HF solution
- We can approximate the ionization energy (IE) and electron affinity (EA) using [Koopman's theorem](https://en.wikipedia.org/wiki/Koopmans%27_theorem) (technically Koopman didn't mention EAs in his orin
- We can check our answer for the IE against the one from [NIST](https://webbook.nist.gov/cgi/cbook.cgi?ID=C71432&Mask=20) (9.24378 eV) 
- We can check our answer for the EA against the one from [Jordan et al](https://pubs.acs.org/doi/pdf/10.1021/ja00439a014) (-1.15 eV)

In [6]:
ie_nist = 9.24378 # NIST value in eV
ea_jordan = -1.15
ha_to_ev = 27.114 # eV/hartrees

print("Ionization Energy Predicted by Koopman's Theorem = %.4f (eV)" % (-mf.mo_energy[20]*ha_to_ev))
print("Error in Predicted Ionization Energy = %.4f (eV)" % abs(ie_nist+mf.mo_energy[20]*ha_to_ev))
print('\n')
print("Electron Affinity Predicted by Koopman's Theorem = %.4f (eV)" % (-mf.mo_energy[21]*ha_to_ev))
print("Error in Predicted Electron Affinity = %.4f (eV)" % abs(ea_jordan+mf.mo_energy[21]*ha_to_ev))

Ionization Energy Predicted by Koopman's Theorem = 9.0088 (eV)
Error in Predicted Ionization Energy = 0.2350 (eV)


Electron Affinity Predicted by Koopman's Theorem = -3.6651 (eV)
Error in Predicted Electron Affinity = 2.5151 (eV)


- We can see here that the IE is not a bad approximation, but the electron affinity is quite bad, even predicting the sign incorrectly.

In [7]:
print("HF Excitation Energy %.4f" % ((mf.mo_energy[21]-mf.mo_energy[20])*ha_to_ev))
print(mf.mo_energy[20], mf.mo_energy[21])

HF Excitation Energy 12.6738
(-0.33225489829901395, 0.13517214063329305)


In [8]:
# Cleanup Dice Files
import os
os.system('rm *bkp')
os.system('rm *dat')
os.system('rm *txt')
os.system('rm shci.e')
os.system('rm FCIDUMP')
os.system('rm tmp*')


0